## Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## Configuration Manager

In [ ]:
from textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummarizer.utils.common import create_directories, read_yaml
from textSummarizer.entity import DataIngestionConfig


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([self.config.artifacts_root])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

## Component

In [ ]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
from pathlib import Path
from textSummarizer.entity import DataIngestionConfig


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    def initiate_data_ingestion(self):
        self.download_file()
        self.extract_zip_file()

## Pipeline

In [6]:
from textSummarizer.config.configuration import ConfigurationManager
from textSummarizer.components.data_ingestion import DataIngestion
from textSummarizer.logging import logger


config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.initiate_data_ingestion()


[2023-10-10 10:33:22,313:  textSummarizerLogger: INFO: common: Line 28 - yaml file: config/config.yaml loaded successfully]
[2023-10-10 10:33:22,327:  textSummarizerLogger: INFO: common: Line 28 - yaml file: params.yaml loaded successfully]
[2023-10-10 10:33:22,331:  textSummarizerLogger: INFO: common: Line 46 - created directory at: artifacts]
[2023-10-10 10:33:22,333:  textSummarizerLogger: INFO: common: Line 46 - created directory at: artifacts]
[2023-10-10 10:33:22,334:  textSummarizerLogger: INFO: data_ingestion: Line 22 - File already exists of size: ~ 8 MB]


## Main

In [7]:
from textSummarizer.pipeline.stage_01_data_ingestion import DataIngestionTrainingPipeline
from textSummarizer.logging import logger

STAGE_NAME = "Data Ingestion stage"
try:
    logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
    data_ingestion = DataIngestionTrainingPipeline()
    data_ingestion.main()
    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
    logger.exception(e)
    raise e


[2023-10-10 10:33:33,326:  textSummarizerLogger: INFO: 367653918: Line 6 - >>>>>> stage Data Ingestion stage started <<<<<<]
[2023-10-10 10:33:33,328:  textSummarizerLogger: INFO: common: Line 28 - yaml file: config/config.yaml loaded successfully]
[2023-10-10 10:33:33,330:  textSummarizerLogger: INFO: common: Line 28 - yaml file: params.yaml loaded successfully]
[2023-10-10 10:33:33,330:  textSummarizerLogger: INFO: common: Line 46 - created directory at: artifacts]
[2023-10-10 10:33:33,330:  textSummarizerLogger: INFO: common: Line 46 - created directory at: artifacts]
[2023-10-10 10:33:33,331:  textSummarizerLogger: INFO: data_ingestion: Line 22 - File already exists of size: ~ 8 MB]
[2023-10-10 10:33:33,410:  textSummarizerLogger: INFO: 367653918: Line 9 - >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
